# Detecção de fakenews

- Baseado em https://www.youtube.com/watch?v=UMw5-PhRZS0
- dataset https://github.com/recogna-lab/datasets/tree/master/FakeRecogna

In [17]:
#importando bibliotecas
import pandas as pd 
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
print(stopwords.words('portuguese'))

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [20]:
#dataset
df = pd.read_excel('./dataset/FakeRecogna.xlsx')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11903 entries, 0 to 11902
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Titulo     11872 non-null  object 
 1   Subtitulo  5580 non-null   object 
 2   Noticia    11902 non-null  object 
 3   Categoria  11902 non-null  object 
 4   Data       11551 non-null  object 
 5   Autor      11886 non-null  object 
 6   URL        11902 non-null  object 
 7   Classe     11902 non-null  float64
dtypes: float64(1), object(7)
memory usage: 744.1+ KB


In [22]:
df = df[['Titulo', 'Autor', 'Noticia', 'URL', 'Classe']]

In [23]:
#verificando os nulos
df.isnull().sum()

Titulo     31
Autor      17
Noticia     1
URL         1
Classe      1
dtype: int64

In [24]:
#retirando os nulos
df.dropna(inplace=True)

In [25]:
#verificando os nulos novamente
df.isnull().sum()

Titulo     0
Autor      0
Noticia    0
URL        0
Classe     0
dtype: int64

In [38]:
len(df)

11856

In [26]:
#juntando os campos
df['all']=df['Titulo']+' '+df['Autor']+' '+df['Noticia']

In [27]:
#verificando a junção dos campos
pd.options.display.max_colwidth = 150

print(df['all'])

0        \n\nPapa Francisco foi preso sob acusação de tráfico de crianças e fraudes #boato \n \nEdgard Matsuki  apagão vaticano papar presar acusação tráfi...
1        Equador prepara cova coletiva para mortos por covid-19 em Guaiaquil   27/03/2020 18h25  o governar equador anunciar preparar cova coletiva cemitér...
2        Air France voltará a operar voo direto Pequim-Paris   07/08/2020 13h42  o companhia air france operar voar direto pequim e parir agostar o o capit...
3        Marfrig intensifica venda de carne do Brasil aos EUA após fechamentos por covid-19   27/04/2020 14h53  o marfrig global foods retomar vender carna...
4        As parciais das eleições de 2014 alternaram mais de 200 vezes a vitória entre Aécio e Dilma? Gilmar Lopes o assunto voltar o compartilhar rede soc...
                                                                                 ...                                                                          
11898    \n\nArgentina vacinou duas vezes o me

In [28]:
#criando a função de steming
porter_stemmer = PorterStemmer()

def stemming_tokenizer(df):

    words = re.sub(r"[^A-Za-z0-9]", " ", df).lower().split()
    words = [porter_stemmer.stem(word) for word in words if not words in stopwords.words('portuguese')]
    words = ' '.join(words)
    return words

In [29]:
df['features'] = df['all'].apply(stemming_tokenizer)

In [30]:
print(df['features'])

0        papa francisco foi preso sob acusa o de tr fico de crian as e fraud boato edgard matsuki apag o vaticano papar presar acusa o tr fico crian a e fr...
1        equador prepara cova coletiva para morto por covid 19 em guaiaquil 27 03 2020 18h25 o governar equador anunciar preparar cova coletiva cemit rio e...
2        air franc voltar a operar voo direto pequim pari 07 08 2020 13h42 o companhia air franc operar voar direto pequim e parir agostar o o capit chin s...
3        marfrig intensifica venda de carn do brasil ao eua ap s fechamento por covid 19 27 04 2020 14h53 o marfrig global food retomar vender carnar bovin...
4        as parciai da elei es de 2014 alternaram mai de 200 veze a vit ria entr a cio e dilma gilmar lope o assunto voltar o compartilhar rede social julh...
                                                                                 ...                                                                          
11898    argentina vacin dua veze o mesmo home

In [31]:
#separando em features e "labels"
x = df['features'].values
y = df['Classe'].values

In [32]:
#convertendo de texto para numeros
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

TfidfVectorizer()

In [33]:
x = vectorizer.transform(x)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2, stratify=y, random_state=42 )

In [35]:
#treinando o modelo
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [36]:
#previsão
pred = lr.predict(X_test)

acc = accuracy_score(y_test, pred)

f'acurácia:{ acc * 100:.2f}'

'acurácia:96.59'

In [37]:
import pickle

pickle.dump(lr, open("lrModel", "wb"))
pickle.dump(vectorizer, open("vectorizer", "wb"))

In [20]:
model = pickle.load(open("lrModel", "rb"))
model = pickle.load(open("vectorizer", "rb"))

In [21]:
import newspaper

def getNewsArticle(url): 
  article = newspaper.Article(url, language='pt')
  
  article.download()
  article.parse()  

  return article.title +' '+' '.join(article.authors)+' '+article.text

In [22]:
news = getNewsArticle("https://g1.globo.com/fato-ou-fake/noticia/2023/03/17/e-fake-que-vanderlei-luxemburgo-morreu-em-queda-de-helicoptero-na-barra-funda.ghtml")

In [23]:
sampleDf = pd.DataFrame({ "features": [news] })
sampleDf["features"] = sampleDf["features"].apply(stemming_tokenizer)

In [24]:
sample = vectorizer.transform(sampleDf["features"].values)

In [28]:
bool(model.predict(sample)[0])

False